In [1]:
import threading
import time
import numpy as np

cond = threading.Condition()

class AInt():
    def __init__(self, val):
        self.v = val

done = AInt(0) #shared int

#Representation of a city
class City():
    def __init__(self, label, coordinates, salesman):
        self.label = label
        self.coordinates = coordinates
        self.distance = -1
        self._s = salesman
        self._t = threading.Thread
        self._ok = True

    #It just observes the distance between him and the salesman
    def observe(self, cond, done):
        while self._ok:
            self.distance = np.linalg.norm(self.coordinates - self._s)
            with cond:
                done.v += 1 #atomic
                cond.wait()

cities = {
    "A": np.array([1, 2]),
    "B": np.array([2, 5]),
    "C": np.array([8, 7]),
    "E": np.array([4, 3]),
    "F": np.array([7, 8]),
    "G": np.array([3, 2]),
    "H": np.array([1, 4]),
    "I": np.array([3, 5]),
    "J": np.array([23, 20]),
    "K": np.array([41, 3]),
    "L": np.array([6, 7]),
    "M": np.array([7, 50]),
    "N": np.array([8, 23]),
    "Ñ": np.array([12, 2]),
    "O": np.array([1, 15]),
    "P": np.array([5, 34]),
    "Q": np.array([8, 4]),
    "R": np.array([90, 2]),
    "S": np.array([23, 76]),
    "T": np.array([22, 45]),
    "U": np.array([45, 89]),
    "V": np.array([12, 2]),
    "W": np.array([67, 9]),
    "X": np.array([3, 0]),
    "Y": np.array([8, 13]),
    "Z": np.array([1, 1]),
}

salesman = np.array([20, 42]) #Salesman start point
trace = []
cs = []

#Initialice city thread observers
for k, v in cities.items():
    c = City(k, v, salesman)
    threading.Thread(target=c.observe, args=(cond, done,)).start()
    cs.append(c)


while len(cs) > 0:
    while done.v != len(cs): #wait till all observers has done
        pass

    nearer = cs[0]
    for i, v in enumerate(cs[1:]):
        if v.distance < nearer.distance:
            nearer = v

    cs.remove(nearer)
    nearer._ok = False
    salesman[0] = nearer.coordinates[0]
    salesman[1] = nearer.coordinates[1]
    trace.append(nearer.label)

    #Reset the done counter and allow the observers, to observe again
    done.v = 0
    with cond:
        cond.notifyAll()
        
print(trace)

['T', 'M', 'P', 'N', 'Y', 'F', 'C', 'L', 'I', 'B', 'H', 'A', 'Z', 'G', 'E', 'X', 'Q', 'Ñ', 'V', 'O', 'J', 'K', 'W', 'R', 'U', 'S']
